# Keras Neural Network Framework for Classification and Regression
**Multi‐Task**: Classification on Pima Diabetes & Regression on Energy Efficiency

Modify the `TASK` variable in the next cell to switch between modes.

In [ ]:
# Configuration Panel
TASK = 'classification'  # 'classification' or 'regression'
HIDDEN_UNITS = 64
DROPOUT_RATE = 0.5
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 32

## Core Imports
- NumPy, pandas for data
- scikit‐learn for splits & metrics
- standalone Keras for model

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
# silence C++ INFO and WARNING messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'   # 0 = all, 1 = filter INFO, 2 = filter WARNING, 3 = filter ERROR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error

# Keras imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy, MeanSquaredError
from keras.utils import set_random_seed

# fix random seeds
set_random_seed(42)
np.random.seed(42)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping criteria
early_stopping = EarlyStopping(
    monitor='val_loss',    # Metric to monitor (validation loss)
    patience=20,           # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restores model weights from the epoch with best performance
)


## Data Loader
- Loads Pima (classification) or Energy (regression)
- Standardizes features
- Returns train/test splits

**Note:** Here the datasets are loaded directly from the web. You can also load the same data using the datasets from the other two exercises.

In [ ]:
def load_data(seed=42):
    if TASK=='classification':
        data = pd.read_csv("../datasets/pima-indians-diabetes.csv", header=None)
        
        # Separate features (all columns except last) and target (last column)
        X = data.iloc[:, :-1].values
        y = data.iloc[:, -1].values
    else:
        data = pd.read_csv('../datasets/ENB2012_data.csv', header=None)
        
        # Feature engineering recommendations would go here
        # (e.g., feature scaling, outlier handling)
        
        # Separate features and targets
        X = data.iloc[:, 0:8].values
        y = data.iloc[:, 8:10].values  # Both heating and cooling loads
    
    return X, y

## Model Builder
- Creates a Sequential Keras model
- Adjusts last layer & loss for task 
- `HIDDEN_UNITS` is the width of the first hidden layer, say 64 neurons.  Using `HIDDEN_UNITS//2` (i.e.\ 32) in the next layer creates a “funnel” architecture:  
     $$
       8\;\xrightarrow{\rm relu}\;64\;\xrightarrow{\rm relu}\;32\;\xrightarrow{}1
     $$
  This reduces parameter count (fewer weights → less overfitting) and encourages the network to compress features gradually.  You could use the same width twice, but halving is a common heuristic.

In [ ]:
def build_model(input_dim):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))
    model.add(Dense(HIDDEN_UNITS, activation='relu'))
    model.add(Dropout(DROPOUT_RATE))
    model.add(Dense(HIDDEN_UNITS//2, activation='relu'))
    
    if TASK=='classification':
        model.add(Dense(1, activation='sigmoid'))
        loss = BinaryCrossentropy()
        metrics = ['accuracy']
    else:
        model.add(Dense(2, activation='linear'))
        loss = MeanSquaredError()
        metrics = ['mse']
    model.compile(optimizer=Adam(LEARNING_RATE), loss=loss, metrics=metrics)
    return model

## Training & Evaluation
- Fits model on train split
- Evaluates on test split
- Plots training curves

In [ ]:
def train_and_eval(seed=42):
    X, y = load_data(seed=seed)

    # Splitting data
    Xtr,Xte,ytr,yte = train_test_split(X,y,test_size=0.3,random_state=seed)

    # Scaling the daatset
    scaler = StandardScaler().fit(Xtr)
    Xtr = scaler.transform(Xtr)
    Xte = scaler.transform(Xte)

    # build the model
    mdl = build_model(Xtr.shape[1])
    hist = mdl.fit(Xtr,ytr,
                   validation_split=0.2, # data is split into training and validation
                   epochs=EPOCHS,
                   batch_size=BATCH_SIZE,
                   callbacks=[early_stopping],  # Add the callback here for early stopping. Remove this line for no early stopping.
                   verbose=1)
    print("\nEvaluation:")

    # prediction on test data
    if TASK=='classification':
        ypred = (mdl.predict(Xte)>0.5).astype(int)
        print("Accuracy:",accuracy_score(yte,ypred))
    else:
        ypred = mdl.predict(Xte)
        print("MSE:",mean_squared_error(yte,ypred))
        
    # plot
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(hist.history['loss'],label='train')
    plt.plot(hist.history['val_loss'],label='val'); plt.legend(); plt.title('Loss')
    if TASK=='classification':
        plt.subplot(1,2,2)
        plt.plot(hist.history['accuracy'],label='train')
        plt.plot(hist.history['val_accuracy'],label='val'); plt.legend(); plt.title('Accuracy')
    else:
        plt.subplot(1,2,2)
        plt.plot(hist.history['mse'],label='train')
        plt.plot(hist.history['val_mse'],label='val'); plt.legend(); plt.title('MSE')
    plt.tight_layout(); plt.show()

## Run Everything

In [ ]:
seed = 0
train_and_eval(seed=seed)

# Homework challenges

- Run the code for classification and regression
- Change the learning rate and verify the performance in both the modes
- Increase and descrease the number layers to see the performance change
- Understand how dropout works by change the dropout rate. `DROPOUT_RATE = 0.0` means no dropout
- Modify the code to run the experiments independently multiple times (changing the seed value) to compute the average performance